In [46]:
import pandas as pd
df = pd.read_csv("hc_zip_codes.csv")
df = df.applymap(str.strip)
# df["Zip"] = df["Zip"].astype(pd.StringDtype())
df["End"] = df["Zip"].str[4:].astype("string")
df["Code"] = df["Zip"].str[:4]
df["Code"] = pd.to_numeric(df["Code"], errors="coerce")
df = df.dropna(subset=["Code"])
df["Code"] = df["Code"].astype(int)

df.dtypes

Zip     object
End     string
Code     int32
dtype: object

In [47]:
#3000 AA - 3099 ZZ: Rotterdam (https://en.youbianku.com/netherlands)

df_rot = df.pipe(lambda x: x.loc[(x['Code'] <= 3099) & (x["Code"] >= 3000)])
# df_rot = df[int(df.Zip.str[:3]) == 777]
df_rot


,Zip,End,Code
15,3081BX,BX,3081
31,3082KG,KG,3082
47,3073EM,EM,3073
59,3072VR,VR,3072
107,3081RR,RR,3081
...,...,...,...
1371,3014KD,KD,3014
1377,3083NG,NG,3083
1380,3067DA,DA,3067
1392,3022TL,TL,3022


In [55]:
physical_locations = pd.read_csv("NL_full.txt", header=None, delimiter=r"\s+", on_bad_lines="skip")
physical_locations.columns
d = physical_locations.iloc[:, [1, 2, 8, 9]]
column_names = ["Code", "End", "Lat", "Lon"]
d = d.set_axis(column_names, axis=1)
d.head(5)

,Zip,End,Lat,Lon
0,9401,AB,52.9984,6.5661
1,9401,AC,52.9989,6.5687
2,9401,AD,52.9982,6.5659
3,9401,AE,52.9981,6.5669
4,9401,AG,52.9976,6.5684
